# Ön İşleme (Preprocessing) ve Frekans Bulma


### Kütüphaneleri dahil edelim
`nltk`: NLTK(Natural Language Toolkit(Doğal Dil Kiti)), insan dili verileriyle çalışmak için Pyhton ile geliştirilmiş açık kaynaklı bir kütüphanedir.

`getcwd`: Dosyanın çalıştığı adresi getirir

`numpy`: Numerical Python, çok boyutlu dizilerle ve matrislerle çalışmamızı sağlayan ve matematiksel işlemler yapabileceğimiz Python dili kütüphanelerindendir. 

`pandas`: Veri işlemesi ve analizi için Python ile yazılmış olan bir kütüphanedir.

In [1]:
import re
import string

from os import getcwd
import numpy as np
import pandas as pd

import nltk 
from nltk.corpus import twitter_samples 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

### Tweet örneklerini ve stopwordleri indirelim

In [2]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/macbookair/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbookair/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Veri Seti
İndirdiğimiz veri seti 5000 olumlu 5000 olumsuz tweet içeriyor.

In [3]:
tum_poz_tweetler = twitter_samples.strings('positive_tweets.json')
tum_neg_tweetler = twitter_samples.strings('negative_tweets.json')

Aşağıdaki kod parçacığı sayesinde çalışma alanımızı yenilediğimizde  dosyaları tekrar indirmeden içe aktarılmasını sağlarız

In [4]:
dosyaAdr = f"{getcwd()}/../tmp2/"
nltk.data.path.append(dosyaAdr)

### Oluşturacağımız fonksiyonlar:
* `tweet_isle()`: Metni temizleyip, kelimelere ayırıp (tokenization), stopwordleri kaldırıp, kökleme (stemming) yapacağımız fonksiyon. Yani tüm ön işleme burada olacak.
* `frekans_olustur()`: Kelimelerin olumlu/olumsuz tweetlerdeki frekanslarını hesaplayacağımız fonksiyon.


In [5]:
def tweet_isle(tweet):
    # Girdi olarak islenmemis bir tweet aliyoruz

    # Hashtagleri, linkleri vb. ibareleri tweetten temizliyoruz
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r':()', '', tweet)
    
    # Metni kelimelere ayiriyoruz
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweetin_kelimeleri = tokenizer.tokenize(tweet)
    
    ing_stopwordler = stopwords.words('english')
    
    # Sonucu dondurecegimiz diziyi bos olarak olusturuyoruz
    islenmis_tweet = []
    
    for kelime in tweetin_kelimeleri:
        if (kelime not in ing_stopwordler and kelime not in string.punctuation and not bool(re.search(r'\d', kelime))):  
            # Kelime stopword veya noktalama isareti degilse ve rakam icermiyorsa:
            kelime_koku = PorterStemmer().stem(kelime)  
            # Kokleme yapiliyor ve diziye ekleniyor
            islenmis_tweet.append(kelime_koku)

    return islenmis_tweet

### İlk fonksiyonu deneyelim

In [6]:
print('Bir pozitif cumle: \n', tum_poz_tweetler[1300])
print('\nPozitif cumlenin islenmis hali: \n', tweet_isle(tum_poz_tweetler[1300]))

Bir pozitif cumle: 
 OMG so I did some math and if my channel continues like this (i doubt  it) I will get 2 300 subs when my 200 subs video comes out :) :/

Pozitif cumlenin islenmis hali: 
 ['omg', 'math', 'channel', 'continu', 'like', 'doubt', 'get', 'sub', 'sub', 'video', 'come']


In [7]:
def frekans_olustur(tweetler, duygular):
    # Input olarak tweetler ve duygularini aliyoruz
    # Duygu etiketlerini list haline getiriyoruz
    duygu_listesi = np.squeeze(duygular).tolist()

    # Kelimenin hangi duygudaki kelimede gectigine gore frekansini bir arttiriyoruz
    frekanslar = {}
    for duygu, tweet in zip(duygu_listesi, tweetler):
        for kelime in tweet_isle(tweet):
            ikili = (kelime, duygu)
            if ikili in frekanslar:
                frekanslar[ikili] += 1
            else:
                frekanslar[ikili] = 1

    return frekanslar

### İkinci fonksiyonu deneyelim

In [8]:
tum_tweetler = tum_poz_tweetler + tum_neg_tweetler

duygular = np.append(np.ones((5000, 1)), np.zeros((5000, 1)), axis=0)

frekanslar = frekans_olustur(tum_tweetler, duygular)


In [9]:
print( "'Rain' kelimesi olumlu cumlelerde " , frekanslar.get(("rain", 1.0),0) , " defa gecmis.")
print( "'Rain' kelimesi olumsuz cumlelerde ", frekanslar.get(("rain", 0.0),0)," defa gecmis.")

'Rain' kelimesi olumlu cumlelerde  21  defa gecmis.
'Rain' kelimesi olumsuz cumlelerde  43  defa gecmis.


In [10]:
print( "'Hope' kelimesi olumlu cumlelerde ", frekanslar.get(("hope", 1.0),0)," defa gecmis.")
print( "'Hope' kelimesi olumsuz cumlelerde ", frekanslar.get(("hope", 0.0),0)," defa gecmis.")

'Hope' kelimesi olumlu cumlelerde  141  defa gecmis.
'Hope' kelimesi olumsuz cumlelerde  102  defa gecmis.
